In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib as mpl
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim
from collections import defaultdict
import string

In [2]:
# Read data
df=pd.read_csv('C:/Users/yongl/Fordham/Spring 2020/Text Analytics/Text project/customer-support-on-twitter/twcs/twcs.csv')

#preview data
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [3]:
# join request and response into the same row: 'text_x' is the response from customer services; 'text_y' is the request
def pre(df,author):
    a=df[df.author_id==author]
    a=a.merge(df.loc[:,['tweet_id','text']],left_on='in_response_to_tweet_id',right_on='tweet_id')
    a=a[a.response_tweet_id.isnull()]    
    a['text_x']=a.text_x.apply(lambda x: x[:x.find('@')]+x[x.find('@')+8:])    
    return a

In [5]:
amazon = []
amazon.append(pre(df,'AmazonHelp'))
amazon

[        tweet_id_x   author_id  inbound                      created_at  \
 2              275  AmazonHelp    False  Wed Nov 22 10:06:26 +0000 2017   
 3              324  AmazonHelp    False  Wed Nov 22 09:06:00 +0000 2017   
 6              620  AmazonHelp    False  Tue Oct 31 22:28:34 +0000 2017   
 8              625  AmazonHelp    False  Tue Oct 31 22:34:32 +0000 2017   
 10             629  AmazonHelp    False  Wed Nov 01 12:53:34 +0000 2017   
 ...            ...         ...      ...                             ...   
 168818     2987819  AmazonHelp    False  Wed Nov 22 06:19:27 +0000 2017   
 168819     2987821  AmazonHelp    False  Wed Nov 22 06:17:26 +0000 2017   
 168820     2987900  AmazonHelp    False  Wed Nov 22 07:09:54 +0000 2017   
 168821     2987902  AmazonHelp    False  Wed Nov 22 07:09:53 +0000 2017   
 168822     2987938  AmazonHelp    False  Wed Nov 22 07:49:52 +0000 2017   
 
                                                    text_x response_tweet_id  \
 2    

In [6]:
# append all the questions about amazon to a list
q = amazon[-1]['text_y'].to_list()
q

['@AmazonHelp こちらこそありがとうございました。',
 'amazonプライムビデオ、再生エラーが多いです',
 '@115823 I want my amazon payments account CLOSED.  dm me please.',
 '@AmazonHelp Okay, danke für die Info',
 '@AmazonHelp @115826 Yeah this is crazy we’re less than a week away and still no Shipping Information on something that we Pre-ordered back in August',
 '@115830 my package was ‘accidentally’ opened.. 4 items missing worth £97.\nYou need better delivery drivers!! https://t.co/f6SaVBSMqM',
 '@115821 @AmazonHelp why is my order at my local courier for the last 6 days and still hasn’t been delivered to me?? Over 1 week late 😡',
 'Thanks for the style advice, @115833 look ...I think? #Halloween2017 #flamingo https://t.co/XvI54La043',
 '@AmazonHelp Hi ready for some help',
 '@AmazonHelp That page is useless - doesn’t allow me to state it hasn’t been delivered; only tells me it has! How can you sort this out?',
 'In response to your @115830 packing video, this packaging was for a 2ft washing line pole @115837 https://t.c

In [7]:
#remove '\r' and punctuations 
new_text = []
for tweet in q: 
    for i in tweet:
        if i in string.punctuation or i == '@':
            tweet = tweet.replace(i,'') #replace punctuation with nothing
        if i == '\r':
            tweet = tweet.replace(i,' ') #replace \r with space
    new_text.append(tweet) 
new_text

['AmazonHelp こちらこそありがとうございました。',
 'amazonプライムビデオ、再生エラーが多いです',
 '115823 I want my amazon payments account CLOSED  dm me please',
 'AmazonHelp Okay danke für die Info',
 'AmazonHelp 115826 Yeah this is crazy we’re less than a week away and still no Shipping Information on something that we Preordered back in August',
 '115830 my package was ‘accidentally’ opened 4 items missing worth £97\nYou need better delivery drivers httpstcof6SaVBSMqM',
 '115821 AmazonHelp why is my order at my local courier for the last 6 days and still hasn’t been delivered to me Over 1 week late 😡',
 'Thanks for the style advice 115833 look I think Halloween2017 flamingo httpstcoXvI54La043',
 'AmazonHelp Hi ready for some help',
 'AmazonHelp That page is useless  doesn’t allow me to state it hasn’t been delivered only tells me it has How can you sort this out',
 'In response to your 115830 packing video this packaging was for a 2ft washing line pole 115837 httpstcoX21SQHgC0K',
 'AmazonHelp Already started the ret

In [8]:
#remove stop words, to lowercase and tokenize
mystopwords = stopwords.words()
tokens_list = [[word for word in tweet.lower().split(' ') if word not in mystopwords and word.isalpha() and word != 'amazon' and word != 'amazonhelp']
         for tweet in new_text]
tokens_list

[[],
 [],
 ['payments', 'account', 'closed', 'dm', 'please'],
 ['okay', 'danke', 'info'],
 ['yeah',
  'crazy',
  'less',
  'week',
  'away',
  'still',
  'shipping',
  'information',
  'something',
  'preordered',
  'back',
  'august'],
 ['package',
  'opened',
  'items',
  'missing',
  'worth',
  'need',
  'better',
  'delivery',
  'drivers'],
 ['order',
  'local',
  'courier',
  'last',
  'days',
  'still',
  'delivered',
  'week',
  'late'],
 ['thanks', 'style', 'advice', 'look', 'think', 'flamingo'],
 ['hi', 'ready', 'help'],
 ['page', 'useless', 'allow', 'state', 'delivered', 'tells', 'sort'],
 ['response', 'packing', 'video', 'packaging', 'washing', 'line', 'pole'],
 ['already',
  'started',
  'return',
  'ups',
  'gets',
  'doorstep',
  'tomorrow',
  'order'],
 ['already', 'handled', 'venting', 'pretty', 'big', 'bummer'],
 ['feliz'],
 ['delivery',
  'paid',
  'arrivewhy',
  'noti',
  'paid',
  'enough',
  'itwhere',
  'unhappyrefund',
  'delivery',
  'charge'],
 ['yeah',
  'idk'

In [9]:
#remove words that appear only once
frequency = defaultdict(int)

for tokens in tokens_list:
    for token in tokens:
        frequency[token] += 1      
tokens_list = [[token for token in tokens if frequency[token]>1]
              for tokens in tokens_list]
tokens_list

[[],
 [],
 ['payments', 'account', 'closed', 'dm', 'please'],
 ['okay', 'danke', 'info'],
 ['yeah',
  'crazy',
  'less',
  'week',
  'away',
  'still',
  'shipping',
  'information',
  'something',
  'preordered',
  'back',
  'august'],
 ['package',
  'opened',
  'items',
  'missing',
  'worth',
  'need',
  'better',
  'delivery',
  'drivers'],
 ['order',
  'local',
  'courier',
  'last',
  'days',
  'still',
  'delivered',
  'week',
  'late'],
 ['thanks', 'style', 'advice', 'look', 'think'],
 ['hi', 'ready', 'help'],
 ['page', 'useless', 'allow', 'state', 'delivered', 'tells', 'sort'],
 ['response', 'packing', 'video', 'packaging', 'washing', 'line', 'pole'],
 ['already',
  'started',
  'return',
  'ups',
  'gets',
  'doorstep',
  'tomorrow',
  'order'],
 ['already', 'handled', 'venting', 'pretty', 'big', 'bummer'],
 ['feliz'],
 ['delivery', 'paid', 'noti', 'paid', 'enough', 'delivery', 'charge'],
 ['yeah',
  'idk',
  'shipping',
  'company',
  'says',
  'signed',
  'mail',
  'room',


In [10]:
#Generate Term Document Matrix
# generate token dictionary class
dictionary = corpora.Dictionary(tokens_list) 
dictionary

In [11]:
# generate a unique token list 
sort_token = sorted(dictionary.items(),key=lambda k:k[0], reverse = False)
unique_token = [token for (ID,token) in sort_token]

# build a corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokens_list]
corpus

[[],
 [],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1)],
 [(8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1)],
 [(17, 1),
  (18, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1)],
 [(36, 1), (37, 1), (38, 1), (39, 1), (40, 1)],
 [(41, 1), (42, 1), (43, 1)],
 [(31, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1)],
 [(50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1)],
 [(35, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1)],
 [(57, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1)],
 [(69, 1)],
 [(21, 2), (70, 1), (71, 1), (72, 1), (73, 2)],
 [(15, 1),
  (19, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1)],
 [],
 [(32, 1),
  (57, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86

In [12]:
# Save a Term Document Matrix
matrix = gensim.matutils.corpus2dense(corpus,num_terms=len(dictionary),dtype = 'int')
matrix = matrix.T 
#transpose the matrix 

#convert the numpy matrix into pandas dataframe
matrix_df = pd.DataFrame(matrix, columns=unique_token)
matrix_df

,account,closed,dm,payments,please,danke,info,okay,august,away,...,invoiced,refundevery,fmub,ittw,sufb,customertracking,morevever,iwatch,refundbooked,gobi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84561,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84562,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
84564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#fit to LDA model
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=10) 

#Topic matrix (V matrix)
lda.print_topics(10) 

[(0,
  '0.097*"service" + 0.086*"customer" + 0.017*"worst" + 0.015*"disappointed" + 0.015*"support" + 0.014*"bad" + 0.012*"experience" + 0.011*"time" + 0.011*"even" + 0.011*"really"'),
 (1,
  '0.021*"done" + 0.017*"fire" + 0.014*"alexa" + 0.014*"mail" + 0.012*"front" + 0.011*"tv" + 0.011*"mal" + 0.011*"thought" + 0.010*"sold" + 0.010*"anyone"'),
 (2,
  '0.037*"please" + 0.035*"order" + 0.026*"help" + 0.021*"prime" + 0.014*"account" + 0.014*"need" + 0.014*"india" + 0.012*"cancel" + 0.012*"get" + 0.011*"available"'),
 (3,
  '0.055*"order" + 0.034*"received" + 0.026*"still" + 0.022*"yet" + 0.019*"refund" + 0.017*"product" + 0.016*"ordered" + 0.014*"delivery" + 0.013*"date" + 0.012*"havent"'),
 (4,
  '0.042*"delivered" + 0.027*"today" + 0.026*"package" + 0.024*"thanks" + 0.023*"order" + 0.016*"delivery" + 0.016*"says" + 0.013*"yesterday" + 0.011*"said" + 0.011*"tracking"'),
 (5,
  '0.041*"email" + 0.021*"call" + 0.018*"contact" + 0.015*"times" + 0.014*"details" + 0.013*"sent" + 0.013*"link

In [14]:
# Generate U Matrix for LDA model
corpus_lda = lda[corpus] 
#transform lda model

#convert corpus_lda to numpy matrix
U_matrix_lda = gensim.matutils.corpus2dense(corpus_lda,num_terms=10).T

#write U_matrix into pandas dataframe and output
U_matrix_lda_df = pd.DataFrame(U_matrix_lda)
U_matrix_lda_df

,0,1,2,3,4,5,6,7,8,9
0,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
1,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
2,0.016667,0.016667,0.849990,0.016667,0.016667,0.016668,0.016668,0.016668,0.016668,0.016667
3,0.025004,0.025002,0.352088,0.025002,0.447889,0.025002,0.025003,0.025004,0.025003,0.025002
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.731660,0.000000,0.206780,0.000000
...,...,...,...,...,...,...,...,...,...,...
84561,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
84562,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
84563,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000,0.100000
84564,0.021662,0.021662,0.021662,0.021662,0.021662,0.021662,0.021662,0.805039,0.021662,0.021662


In [15]:
print (matrix_df.shape)
print (U_matrix_lda_df.shape)

(84566, 21273)
(84566, 10)


In [16]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.136253 -0.059114       1        1  15.493586
4     -0.139395 -0.020679       2        1  12.811236
2     -0.094265 -0.014179       3        1  12.158794
6     -0.139373 -0.018930       4        1  11.987387
5     -0.073218  0.057174       5        1  11.067451
3     -0.108529 -0.086554       6        1   9.415853
0     -0.012923 -0.054526       7        1   8.317969
7      0.390438 -0.265190       8        1   6.709438
9      0.070166  0.171562       9        1   6.247921
1      0.243352  0.290435      10        1   5.790371, topic_info=     Category         Freq        Term        Total  loglift  logprob
271   Default  5274.000000     service  5274.000000  30.0000  30.0000
270   Default  4676.000000    customer  4676.000000  29.0000  29.0000
35    Default  9886.000000       order  9886.000000  28.0000  28.0000
21    Default  8044.000000    delivery  8044.000000  27.0000  27.0000
121   Default  3062.000000       email  3062.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
1172  Topic10   215.512543  completely   256.596405   2.6745  -5.1660
598   Topic10   253.941849         wtf   384.102905   2.4352  -5.0019
275   Topic10   281.119019         box  1388.918945   1.2515  -4.9002
562   Topic10   216.261230        love   571.883850   1.8765  -5.1625
19    Topic10   196.906433        yeah   271.670685   2.5271  -5.2562

[621 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
237       1  0.001242   able
237       2  0.006209   able
237       4  0.453242   able
237       5  0.532714   able
237       6  0.003725   able
...     ...       ...    ...
465       6  0.706779    yet
465       9  0.027164    yet
3856      1  0.995920  youre
2065      8  0.997457     ça
1509     10  0.994907   être

[1468 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 3, 7, 6, 4, 1, 8, 10, 2])